In [20]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
%matplotlib inline

In [21]:
def make_line_points(y1, y2, line):
    if line is None:
        return None
    slope, intercept = line
    # make sure everything is integer as cv2.line requires it
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def draw_lines(image, lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
                
    # add more weight to longer lines
    left_lane  = np.dot(left_weights,  left_lines)/np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    # creating a blank to draw lines on
    line_image = np.copy(image)*0
    # converting lines into point coordinates
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line = make_line_points(y1, y2, left_lane)
    right_line = make_line_points(y1, y2, right_lane)
    if(left_line is not None):
        cv2.line(line_image,(left_line[0][0],left_line[0][1]),(left_line[1][0],left_line[1][1]),(255,0,0),10)
    if(right_line is not None):
        cv2.line(line_image,(right_line[0][0],right_line[0][1]),(right_line[1][0],right_line[1][1]),(255,0,0),10)
        
    return line_image

In [22]:
##Applying Hue,Saturation and Lightness to seperate the yellow and white lines
def select_white_yellow(image):
    converted = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    # white color mask
    lower = np.uint8([  0, 200,   0])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted, lower, upper)
    # yellow color mask
    lower = np.uint8([ 10,   0, 100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(converted, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    return cv2.bitwise_and(image, image, mask = mask)

In [23]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

def process(image):
    
    """Convert image as per hue, saturation, lightness (HSL) to detect yellow and white lines seperately"""
    yImage = select_white_yellow(image)
    
    """Convert image to gray image"""
    gray = cv2.cvtColor(yImage,cv2.COLOR_RGB2GRAY)
    
    """Convert gray image to blur image to reduce noise effect"""
    kernal_size = 3
    #kernal_size is always in odd numbers
    blur_gray = cv2.GaussianBlur(gray,(kernal_size,kernal_size),0)
    #plt.imshow(blur_gray, cmap = 'gray')
    
    """Apply Canny Edge Detection on blur image"""
    low_threshold = 10
    high_threshold = 100
    edges = cv2.Canny(blur_gray,low_threshold,high_threshold)
    #plt.imshow(edges, cmap = 'gray')
    
    """Apply polygon filter on canny detected image"""
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450,320),(480,320),(imshape[1],imshape[0])]],dtype=np.int32)
    cv2.fillPoly(mask,vertices,ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges,mask)
    #plt.imshow(masked_edges)
    
    """Apply Hough transform on masked edges"""
    rho=2
    theta = np.pi/180
    threshold = 15
    min_line_length = 40
    max_line_gap = 20
    line_image = np.copy(image)*0
    lines = cv2.HoughLinesP(masked_edges,rho,theta,threshold,np.array([]),min_line_length,max_line_gap)
    line_image =draw_lines(image, lines)
    
    color_edges = np.dstack((masked_edges,masked_edges,masked_edges))
    #plt.imshow(color_edges)
    line_edges = cv2.addWeighted(color_edges,0.8,line_image,1,0)
    #plt.imshow(line_edges)
    
    """Apply weighted image function to implement all above functions on input image"""
    weightedImage = weighted_img(line_edges,image,α=0.8, β=1., γ=0.)
    
    """Save final image to result function and return results"""
    result = weightedImage
    #plt.imshow(weightedImage)
    return result


In [24]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import time
import timeit

In [25]:
"""Apply the results on video"""
white_output = 'test_videos_output/solidWhiteRight.mp4'
yellow_output = 'test_videos_output/solidYellowLeft.mp4'

clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
clip2 = VideoFileClip("test_videos/solidYellowLeft.mp4").subclip(0,10)

white_clip = clip1.fl_image(process) #NOTE: this function expects color images!!
yellow_clip = clip2.fl_image(process)
%time white_clip.write_videofile(white_output, audio=False)
%time yellow_clip.write_videofile(yellow_output, audio=False)

                                                                                                                       
                                                                                                                    

t:  32%|██████████████████████                                              | 81/250 [07:13<00:07, 22.24it/s, now=None]

                                                                                                                       
                                                                                                                    

t:  32%|██████████████████████                                              | 81/250 [07:13<00:07, 22.24it/s, now=None]

t:  14%|█████████▏                                                          | 17/125 [06:18<00:10, 10.31it/s, now=None]


t:   0%|                                                                             | 0/125 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4






t:   2%|█                                                                    | 2/125 [00:00<00:39,  3.11it/s, now=None]


t:   4%|██▊                                                                  | 5/125 [00:00<00:28,  4.23it/s, now=None]


t:   6%|████▍                                                                | 8/125 [00:00<00:20,  5.61it/s, now=None]


t:   9%|█████▉                                                              | 11/125 [00:01<00:15,  7.34it/s, now=None]


t:  11%|███████▌                                                            | 14/125 [00:01<00:11,  9.27it/s, now=None]


t:  14%|█████████▏                                                          | 17/125 [00:01<00:09, 11.38it/s, now=None]


t:  16%|██████████▉                                                         | 20/125 [00:01<00:07, 13.55it/s, now=None]


t:  18%|████████████▌                                                       | 23/125 [00:01<00:06, 15.72it/s, now=None]


t:  21%|█████████████

Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 6.63 s
Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4






t:   1%|▌                                                                    | 2/250 [00:00<01:18,  3.16it/s, now=None]


t:   2%|█▍                                                                   | 5/250 [00:00<00:57,  4.28it/s, now=None]


t:   3%|██▏                                                                  | 8/250 [00:00<00:42,  5.67it/s, now=None]


t:   4%|██▉                                                                 | 11/250 [00:01<00:32,  7.41it/s, now=None]


t:   5%|███▌                                                                | 13/250 [00:01<00:26,  9.11it/s, now=None]


t:   6%|████▎                                                               | 16/250 [00:01<00:21, 10.97it/s, now=None]


t:   8%|█████▏                                                              | 19/250 [00:01<00:17, 12.84it/s, now=None]


t:   9%|█████▉                                                              | 22/250 [00:01<00:15, 14.60it/s, now=None]


t:  10%|██████▊      

OverflowError: cannot convert float infinity to integer

In [26]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))




t:  32%|██████████████████████                                              | 81/250 [00:17<00:07, 23.16it/s, now=None]

In [27]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [28]:
"""Apply the results on challenge video"""
challenge_output = 'test_videos_output/challenge.mp4'

clip1 = VideoFileClip("test_videos/challenge.mp4").subclip(0,5)


challenge_clip = clip1.fl_image(process) #NOTE: this function expects color images!!

%time challenge_clip.write_videofile(challenge_output, audio=False)

                                                                                                                       
                                                                                                                    

                                                                                                                 


t:  32%|██████████████████████                                              | 81/250 [08:10<00:07, 22.24it/s, now=None]

t:  14%|█████████▏                                                          | 17/125 [07:15<00:10, 10.31it/s, now=None]


                                                                                                                       
                                                                                                                    

                                                                                                                 


t:  32%|██████████████████████                   

Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4







t:   2%|█                                                                    | 2/125 [00:00<00:40,  3.06it/s, now=None]



t:   3%|██▏                                                                  | 4/125 [00:00<00:30,  4.00it/s, now=None]



t:   5%|███▎                                                                 | 6/125 [00:00<00:23,  5.06it/s, now=None]



t:   6%|████▍                                                                | 8/125 [00:01<00:18,  6.21it/s, now=None]



t:   8%|█████▍                                                              | 10/125 [00:01<00:15,  7.35it/s, now=None]



t:  10%|██████▌                                                             | 12/125 [00:01<00:13,  8.38it/s, now=None]



t:  11%|███████▌                                                            | 14/125 [00:01<00:11,  9.46it/s, now=None]



t:  13%|████████▋                                                           | 16/125 [00:01<00:10, 10.53it/s, now=None]



t:  14%|████

t:  98%|█████████████████████████████████████████████████████████████████▍ | 122/125 [00:10<00:00, 13.21it/s, now=None]



t:  99%|██████████████████████████████████████████████████████████████████▍| 124/125 [00:10<00:00, 13.44it/s, now=None]



                                                                                                                       
                                                                                                                    

                                                                                                                 


t:  32%|██████████████████████                                              | 81/250 [08:22<00:07, 22.24it/s, now=None]

t:  14%|█████████▏                                                          | 17/125 [07:27<00:10, 10.31it/s, now=None]


                                                                                                                       
                                     

Moviepy - Done !
Moviepy - video ready test_videos_output/challenge.mp4
Wall time: 12.1 s


In [29]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))